### Vision input output

raw image -> numerical encoding -> model -> output -> predicted output <br>

* input shapes
```[batch_size, width, height, color_channel]``` ex: [None, 224, 224, 3], [32, 224, 224, 3]

batch_size is problem specific depending on hardware

* output shape: number of class [n, ....]

Different input shapes (for different frameworks):
1. shape -> [None, 28, 28, 1] (NHWC)
2. shape -> [None, 1, 28, 28] (NCHW)

##### Path to build
* Get the data ready (convert to tensors)
* Build or pick a model - Pick a loss function, Build a training loop
* Fit the model to the data and make prediction
* Evaluate the model
* Improve through experimentation
* Save and reload the trained model

#### CNN
- Convolutional layer - 'requires input data, filter (feature detector/kernel), featuremap'
`nb`: we have a feature detector, also known as a kernel or a filter, which will move across the receptive fields of the image, checking if the feature is present. This process is known as a convolution.
- Pooling layer
- Fully-connected layer